### Когортный анализ в Pandas

[Алексей Куличевский](https://smysl.io/blog/pandas/)

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D0%BA%D0%B5%D0%B9%D1%81%D1%8B%20%D0%BF%D0%BE%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D1%83%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/1.%20%D0%9A%D0%BE%D0%B3%D0%BE%D1%80%D1%82%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

Прежде всего, вспомним, что такое когорты и как их анализировать.
```
Когорта — это группа людей, которая совершила нужное действие в определенный промежуток времени.

Когортный анализ — это наблюдение за когортами. Выбираем одну или несколько метрик, измеряем их и делаем выводы.
```
Например, социологи могут отслеживать, сколько людей, родившихся в 1980 году, получили высшее образование. Когорта здесь — те, кто родились в 1980 году. Метрика — доля людей с высшим образованием.

Еще пример: маркетологи хотят узнать, сколько заказов и выручки принесли пользователи, совершившие свой первый заказ год назад. Теперь когорта — это прошлогодние покупатели, а метрики — количество заказов и выручка.

Получается, когортный анализ состоит из трех шагов: определения нужного действия и временного промежутка когорты и выбора метрик для отслеживания:

| Действие        | Время           | Метрика  |
| ------------- |:-------------:| -----:|
| Родились         | В 1980 году | % людей с высшим образованием |
| Впервые что-то купили | Год назад      |   Количество заказов и выручка |
| Установили приложение | Неделю назад      |    % пользователей, открывших приложение еще раз |

Когорты можно сравнивать между собой. Например, маркетологи измеряют, сколько заказов обычно делают пользователи в течение месяца после первой покупки и смотрят динамику:

| Месяц        | Клиенты   | Покупок в 1й месяц   |Покупок на клиента   |
| ------------- |:-------------:| -----:|-----:|
| Январь 2018   | 134      | 161                     |1.20 |
| Февраль 2018  | 164      | 194                     |1.18   |
| Март 2018     | **193**      |  **200**          |**1.03**    |

Общее количество клиентов и покупок выросло — приятно. Но, сравнив когорты, видим, что в среднем клиенты стали покупать реже — тревожный знак.

Ок, теперь мы готовы к тому, чтобы научиться делать когортный анализ с помощью Pandas. Для наглядности решим задачу.

### Задача

Допустим, мы работаем в интернет-магазине и хотим понять, сколько заказов и денег клиенты приносят в течение года после первой покупки. Для этого у нас есть данные о заказах:

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/orders.csv'
orders = pd.read_csv(url, index_col='id')

In [ ]:
orders.head()

Каждая строка таблицы `orders` — это покупка. Мы знаем, когда она произошла, кто её сделал и сколько денег она принесла в магазин. Дата заказа лежит в поле `order_date`, номер покупателя — в `customer_id`, а выручка — в `sales`.

Часто бывает, что даты загружаются в виде текста. Преобразуем колонку `order_date` из текста в дату:

In [ ]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

Данные перед нами, теперь можно с ними работать. Начнем с простого: выясним, сколько всего в магазине было покупок и выручки.

Чтобы посчитать общую выручку, просуммируем колонку `sales`:

In [ ]:
orders['sales'].sum()

Количество заказов можно посчитать с помощью этой же колонки, но вместо суммы используем метод `count()`:

Теперь посчитаем обе метрики для каждого пользователя. 

Сгруппируем датафрейм по полю `customer_id`:

In [ ]:
orders.groupby('customer_id')['sales'].agg(['sum', 'count'])

Видим, например, что пользователь `AA-10315 сделал 5 заказов и принес $5563 выручки`.

Идём дальше. Чтобы построить когорты, нам нужно сгруппировать клиентов по дате их первой покупки. В данных нет такого поля, значит, нужно его посчитать.

Чтобы вычислить дату первой покупки каждого пользователя, сгруппируем данные по `customer_id` и найдем минимальное значение поля `order_date`. Результат сохраним в переменную `first_orders`:

In [ ]:
first_orders = orders.groupby('customer_id')['order_date'].agg(first_order=min)
first_orders.head()

Видим, что пользовать `AA-10315 впервые что-то купил 31 марта 2014 года`, а `пользователь AA-10375 — 21 апреля того же года`.

Зная даты первых покупок, можем строить когорты.

Итак, когортами будем считать людей, сделавших первую покупку в тот или иной день. Метрики для отслеживания — количество заказов и выручка.

План такой: сначала добавим дату первой покупки пользователей в таблицу с заказами, затем сгруппируем по датам первой покупки и заказа и, наконец, посчитаем выручку и количество заказов каждой когорты.

Приступим. Добавим дату первой покупки с помощью метода `merge()` и сохраним получившийся датафрейм в переменную `orders_merged`:

In [ ]:
orders_merged = orders.merge(first_orders, 
                             how='inner', 
                             left_on='customer_id', 
                             right_index=True)
orders_merged.head()

В строках получившегося датафрейма всё еще покупки, но теперь в таблице появилась новая колонка: дата первой покупки пользователя.

Агрегируем по дате первой покупки и посчитаем нужные показатели:

In [ ]:
orders_merged.groupby('first_order')['sales'].agg(['sum', 'count'])

Видим, что `клиенты от 3 января 2014 года, всего сделали 9 заказов на $1050.6`. Посмотрим, когда были эти заказы. Для этого добавим к группировке колонку `order_date`:

In [ ]:
orders_merged.groupby(['first_order','order_date'])['sales'].agg(['sum', 'count'])

Ага, `первый заказ этой когорты был 3 января на $16. В следующий раз клиент вернулся почти год спустя и купил что-то ещё, в этот раз на $153. Следующая покупка была уже в апреле 2015 и так далее`.

Когорты готовы, теперь решим задачу.

### Решаем задачу

Напомню, что мы хотим посчитать, сколько в среднем заказов и выручки приносят клиенты в течение года, после первой покупки.

Мы знаем, сколько магазин заработал с каждой когорты за всё время. Уточним метрику: посчитаем показатели за первый год жизни когорты.

Сначала узнаем, сколько дней прошло между первой покупкой и последующим заказом, и удалим те, которые случились `позже 365 дней`. Чтобы посчитать количество дней между заказами, вычтем из колонки `order_date` столбец `first_order`:

In [ ]:
orders_merged['order_date'] - orders_merged['first_order']

Вуаля. Видим, что, например, заказ `131884 случился 455 дней спустя первой покупки`. 

`455 days` — это тип данных под названием `Timedelta`, его специально придумали, чтобы показывать временные промежутки.

Чтобы удалить поздние заказы, добавим условие `<= '365 days'`:

In [ ]:
orders_merged['order_date'] - orders_merged['first_order'] <= '365 days'

Сохраним результат в переменную `year_1_filter`, отфильтруем ненужные заказы из когортного отчета и сохраним результат в переменную `year_1_orders`:

In [ ]:
year_1_filter = orders_merged['order_date'] - orders_merged['first_order'] <= '365 days'
year_1_orders = orders_merged[year_1_filter]
year_1_orders.head()

В датафрейме остались только заказы, сделанные когортами в первый год после первой покупки. Теперь `сгруппируем заказы по дате первой покупки и посчитаем нужные метрики`. Результат сохраним в переменную `cohorts`:

In [ ]:
cohorts = year_1_orders.groupby(['first_order','order_date'])['sales'].agg(['sum', 'count'])
cohorts.head()

Последний шаг: посчитаем, сколько в среднем заказов и приносят клиенты в течение первого года. Для этого сначала просуммируем показатели каждой когорты, а затем усредним значения методом `mean()`:

In [ ]:
cohorts.groupby('first_order').sum().mean()

Готово! `В среднем за первый год когорты делают по 4 заказа и приносят по $1949 долларов`.

Есть много способов улучшить решение, например сгруппировать дневные когорты в недельные или месячные, визуализировать отчет в таблице или на графике. Наконец, интересно разбить когорты по каким-то признакам, например, отделить частных покупателей от компаний — наверняка их показатели существенно отличаются.